# Вариант 4. Квантование модели с TensorFlow Lite

Простая модель Keras для классификации MNIST: конвертация в TensorFlow Lite **без** квантования и **с** квантованием. Сравнение размера файла и времени инференса.

**Установка зависимостей.** Выполните следующую ячейку один раз (если TensorFlow ещё не установлен), затем перезапустите kernel и запустите ноутбук с начала.

In [ ]:
%pip install tensorflow

In [5]:
# Импорты
from __future__ import annotations

import os
import time

import numpy as np
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

## 1. Загрузка MNIST и обучение модели Keras

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="relu", input_shape=(784,)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax"),
])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=3, validation_split=0.1, verbose=1)
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Точность на тесте: {acc:.4f}")

## 2. Конвертация в TFLite без квантования

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_no_quant = converter.convert()
path_no_quant = "mnist_no_quant.tflite"
with open(path_no_quant, "wb") as f:
    f.write(tflite_no_quant)
size_no_quant = os.path.getsize(path_no_quant)
print(f"Размер TFLite (без квантования): {size_no_quant / 1024:.2f} КБ")

## 3. Конвертация в TFLite с квантованием

In [ ]:
converter_q = tf.lite.TFLiteConverter.from_keras_model(model)
converter_q.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant = converter_q.convert()
path_quant = "mnist_quantized.tflite"
with open(path_quant, "wb") as f:
    f.write(tflite_quant)
size_quant = os.path.getsize(path_quant)
print(f"Размер TFLite (с квантованием): {size_quant / 1024:.2f} КБ")

## 4. Время инференса

Запускаем по 1000 предсказаний на тестовых примерах и усредняем время на один пример.

In [ ]:
def measure_inference_time(tflite_path: str, x_sample: np.ndarray, n_runs: int = 1000) -> float:
    """Среднее время инференса (сек) на один пример за n_runs запусков."""
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()[0]
    interpreter.set_tensor(input_details["index"], x_sample[:1].astype(np.float32))
    start = time.perf_counter()
    for _ in range(n_runs):
        interpreter.set_tensor(input_details["index"], x_sample[:1].astype(np.float32))
        interpreter.invoke()
    return (time.perf_counter() - start) / n_runs

time_no_quant = measure_inference_time(path_no_quant, x_test)
time_quant = measure_inference_time(path_quant, x_test)
print(f"Время инференса (без квант.): {time_no_quant*1000:.3f} мс на пример")
print(f"Время инференса (с квант.):  {time_quant*1000:.3f} мс на пример")

## 5. Сравнение размера файла и времени инференса

In [ ]:
print("--- Сравнение до и после квантования ---")
print(f"  Размер:     без квант. {size_no_quant/1024:.2f} КБ  →  с квант. {size_quant/1024:.2f} КБ  (сжатие в {size_no_quant/max(size_quant,1):.2f}x)")
print(f"  Инференс:   без квант. {time_no_quant*1000:.3f} мс  →  с квант. {time_quant*1000:.3f} мс  (ускорение в {time_no_quant/max(time_quant,1e-9):.2f}x)")

In [ ]:
# Проверка точности TFLite-моделей (опционально)
def eval_tflite_accuracy(tflite_path: str, x: np.ndarray, y: np.ndarray) -> float:
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()
    input_ix = interpreter.get_input_details()[0]["index"]
    output_ix = interpreter.get_output_details()[0]["index"]
    correct = 0
    for i in range(len(x)):
        interpreter.set_tensor(input_ix, x[i : i + 1].astype(np.float32))
        interpreter.invoke()
        pred = np.argmax(interpreter.get_tensor(output_ix))
        if pred == y[i]:
            correct += 1
    return correct / len(x)

acc_no = eval_tflite_accuracy(path_no_quant, x_test[:500], y_test[:500])
acc_q = eval_tflite_accuracy(path_quant, x_test[:500], y_test[:500])
print(f"Точность на 500 тестовых (без квант.): {acc_no:.4f}")
print(f"Точность на 500 тестовых (с квант.):  {acc_q:.4f}")